In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
from models import DCGAN, make_losses

In [ ]:
from preprocess_data import make_dataset, get_all_image_filenames

In [ ]:
fnames = get_all_image_filenames()
N=len(fnames)
print("%d images found" % N)

In [ ]:
epochs=50
batch_size=128
lr=1e-4

In [ ]:
image_set = make_dataset(fnames)
tr_set = image_set.shuffle(N).repeat(epochs).batch(batch_size)
tr_feed = tr_set.make_one_shot_iterator().get_next()

In [ ]:
noise_dim=100

In [ ]:
noise_shape=(batch_size, noise_dim)

In [ ]:
birdgan = DCGAN()
generator = birdgan.build_generator(input_shape=(noise_dim,))
discriminator = birdgan.build_discriminator()

In [ ]:
z = tf.random_uniform(noise_shape)


In [ ]:
losses = make_losses(z, tr_feed, generator, discriminator)

In [ ]:
gen_optim = tf.train.AdamOptimizer(lr)
disc_optim = tf.train.AdamOptimizer(lr)

In [ ]:
gen_tr_op = gen_optim.minimize(losses['generator'], var_list=generator.variables)
disc_tr_op = disc_optim.minimize(losses['discriminator'], var_list = discriminator.variables)

In [ ]:
Nbatch = (N//batch_size)*epochs

In [ ]:
step=50

In [ ]:
import time

In [ ]:
batch = 0
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    while True:
        try:
            t=time.time()
            __,__, losses_ = sess.run([disc_tr_op, gen_tr_op, losses])
            print("time for batch: {0:.2e} sec".format(time.time()-t))
        except tf.errors.OutOfRangeError:
            break